# Análise do MF58

In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
using Revise


In [ ]:
includet("fem-thermistor.jl")

In [ ]:
meshfile = "mf58.msh"

In [ ]:
Tw = 200.0  # Temperatura do termistor (interna)
Tref = 20.0 # Temperatura do ar
Rw = 100.0 # Resistência

In [ ]:
# Velocidade do vento
U1 = [0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 3.5, 4, 5, 6,7, 8, 10, 12, 14, 16, 18, 20];
#U1 = [1.0, 3.0, 10.0]
U2 = [(u,0.5*u)  for u in U1];
U3 = [(u,2*u)  for u in U1];

In [ ]:
th = [ModelInfo.(U1) ModelInfo.(U2) ModelInfo.(U3)];

In [ ]:
model = model = GmshDiscreteModel("mf58.msh")

In [ ]:
fem = fem_model.(th, [model]);

In [ ]:
# Escrever 
dname = "vtk"
if !isdir(dname)
    mkdir(dname)
end
numstring(x,n=3) = string(x + 10^n)[2:end]
uname = numstring.(round.(Int, 10*U1))
fnames1 = joinpath.(dname, "thermistor_" .* uname)
fnames2 = joinpath.(dname, "thermistor_ulo_" .* uname)
fnames3 = joinpath.(dname, "thermistor_uhi_" .* uname)
ferrlo = joinpath.(dname, "err_ulo_" .* uname)
ferrhi = joinpath.(dname, "err_uhi_" .* uname)

nfem = length(U1)
for i in 1:nfem
    T = fem[i,1][1] + Tref
    Thi = fem[i,2][1] + Tref
    Tlo = fem[i,3][1] + Tref
    
    errlo = Tlo - T
    errhi = Thi - T
    writevtk(fem[i,1][2][1], fnames1[i], cellfields=["T"=>T, "Thi"=>Thi, "Tlo"=>Tlo, "errhi"=>errhi, "errlo"=>errlo])
    #writevtk(fem[i,1][2][1], fnames1[i], cellfields=["T"=>fem[i,1][1]+Tref])
    #writevtk(fem[i,2][2][1], fnames2[i], cellfields=["T"=>fem[i,2][1]+Tref])
    #writevtk(fem[i,3][2][1], fnames3[i], cellfields=["T"=>fem[i,3][1]+Tref])#

    #writevtk(fem[i,1][2][1], ferrlo[i], cellfields=["T"=>fem[i,2][1] - fem[i,1][1]])
    #writevtk(fem[i,1][2][1], ferrhi[i], cellfields=["T"=>fem[i,3][1] - fem[i,1][1]])
    
end    

In [ ]:
e

In [ ]:
th[1,1].k

In [ ]:
x = process_thermistor.(th, fem);

In [ ]:
size(x)

In [ ]:
surftemp(x,i=1) = x[1][i]
surfheat(x,i=1,k=2) = x[k][i]


In [ ]:
Tg = surftemp.(x,1) .+ Tref; # Temp. superf. do vidro
Ts = surftemp.(x,2) .+ Tref; # Temp. superf do fio

In [ ]:
Qhg = surfheat.(x,1,2); # Transf. calor do vidro (conv)
Qhs = surfheat.(x,2,2); # Transf. calor do fio (conv)

In [ ]:
# Transferência de calor a partir da superfíce quente no interior
Qt = -surfheat.(x,1,3) - surfheat.(x,2,3);

In [ ]:
Qhg1 = surfheat.(x,1,4); # Transf. calor do vidro (cond)
Qhs1 = surfheat.(x,2,4); # Transf. calor do fio (cond)

In [ ]:
# Transferência de calor total
Qh = Qhg + Qhs;
Qh1 = Qhg1 + Qhs1; 

In [ ]:
# Tensão no termistor:
E = sqrt.( Qh .* Rw);

In [ ]:
E[:,1]

Condição ideal

 * A superfície externa do vidro tem temperatura do termistor (Tw).
 * Toda a transferência de calor se dá no vidro


In [ ]:
Ag = 2*x[1][5][1]  # Area da região de vidro
hg = (w->w.h["hglass"]).(th)
Qi = hg .* Ag .* (Tw - Tref);
Ei = sqrt.( Qi .* Rw);

In [ ]:
Ei[:,1]

In [ ]:
using CairoMakie
using CurveFit

In [ ]:
#fit0 = PowerFit(U1, Ei[:,1].^2)
#fit1 = PowerFit(U1, E[:,1].^2)
#fit2 = PowerFit(U1, E[:,2].^2)

fit0 = PowerFit(Ei[:,1], U1)
fit1 = PowerFit(E[:,1], U1)
fit2 = PowerFit(E[:,2], U1)
fit3 = PowerFit(E[:,3], U1)

In [ ]:
fit1

In [ ]:
fig = Figure()
ax = Axis(fig[1,1])
ei = Ei[:,1]; e1 = E[:,1]; e2 = E[:,2]; e3 = E[:,3]

scatter!(ax, U1, ei, colr=:black, label="Ideal")
lines!(ax, fit0.(ei), ei, color=:black)
scatter!(ax, U1, e1, color=:green, label="Homogeneo")
lines!(ax, fit1.(e1), e1, color=:green)
scatter!(ax, U1, e2, color=:blue, label="Fio Vel Menor")
lines!(ax, fit2.(e2), e2, color=:blue)
scatter!(ax, U1, E[:,3], color=:red, label="Fio Vel Maior")
lines!(ax, fit3.(e3), e3, color=:red)
fig

## Ler os dados experimentais

In [ ]:
using HDF5

In [ ]:
Ec, Uc,Tc= h5open("calibr-data.h5", "r") do h
   
    
    E = read(h["E"])
    U = read(h["U"])
    T = read(h["T"])
    iz = U[:,1] .> 0
    
    6 .* E[iz,:],U[iz,:],T[iz,:]
end

In [ ]:
lines(U1, E[:,1])
scatter!(Uc[:,1], Ec[:,1])
current_figure()

In [ ]:
PowerFit(Ec[:,1], Uc[:,1])

#### labels = get_face_labeling(model)

In [ ]:
tags = get_face_tag(labels, 2);

In [ ]:
get_tag_from_name(labels, "glass")

In [ ]:
ee = 12
fit1(ee), fit2(ee), fit3(ee)

In [ ]:
th .|> getfield

In [ ]:
getfield(th[1], :h)

In [ ]:
king_fit(E[:,3], U1)